In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [11]:
df = pd.read_csv('data/fake_users_data.csv')

In [12]:
df = pd.get_dummies(df, prefix=['age', 'gender', 'type', 'diff', 'comp', 'trans'], columns=['age', 'gender', 'type_route', 'difficulty','accompaniment', 'transport'])

In [13]:
routes_dictionary ={'Arbres monumentals i singulars':0,
                    'Arbres: La volta al món':1,
                    'Arbres: Els indians':2,
                    'Arbres: Un passeig per l’Albereda':3,
                    'Arbres: Un viatge pel Mediterrani':4,
                    'Ruta Carrer Russafa - la Llotja':5,
                    'Contes Blasco Ibañez – Ruta 1':6,
                    'Contes Blasco Ibañez – Ruta 2':7,
                    'Entre Naranjos':8,
                    'Glorieta-Santa Catalina':9,
                    'Jardins del Reial - Glorieta':10,
                    'La barraca':11,
                    'Ruta Cultural Anell Ciclista':12,
                    'Ruta de la Seda':13,
                    "Ruta dels Pecats a l'entorn del Mercat Central":14,
                    'Ruta pel Centre Històric':15,
                    "Ruta recinte enmurallat d'època islàmica":16,
                    'Ruta València amb bicicleta':17,
                    'Sant Agustí – Pont de Fusta':18,
                    'Sant Agustí - Portal Nou':19,
                    'Serrans - Palau de la Generalitat':20,
                    'Torres de Quart - Temple':21,
                    "Trinitat - L'Almodí":22,
                    'Ruta Valencia en la Memòria':23}

In [14]:
df['route_id'] = df['route'].map(routes_dictionary)

In [15]:
X = df.drop(columns=['route_id','route'], axis=1)
y = df[['route_id']]

In [16]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression()

from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=3, random_state=42)

from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, random_state=42)

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
estimator = DecisionTreeClassifier(max_depth=3)
ada = AdaBoostClassifier(base_estimator = estimator, n_estimators=200, learning_rate=0.5, random_state=42)

from sklearn.ensemble import GradientBoostingClassifier
gbct = GradientBoostingClassifier(max_depth=3, n_estimators=100, random_state=42)

from xgboost import XGBRFClassifier
xgb = XGBRFClassifier(random_state=42)

from lightgbm import LGBMClassifier
lgb = LGBMClassifier()

from catboost import CatBoostClassifier
cat = CatBoostClassifier(verbose=0)

In [18]:
models = [log, tree, forest, ada, gbct, cat, lgb, xgb]
names = ['Logistic', 'Decision Tree', 'Random Forest', 'AdaBoost', 'Gradient Boosting', 'CatBoost', 'Lightgbm', 'XGBoost']
scores = []
predictions = {}
for idx, model in enumerate(models):
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    predictions[names[idx]] = y_pred
    scores.append(accuracy_score(y_pred, y_test))

c:\Users\Paris\anaconda3\envs\desafio\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paris\anaconda3\envs\desafio\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Paris\AppData\Local\Temp\ipykernel_4800\2367934231.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), 

In [19]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(random_state=42)
mlp.fit(X_train, y_train)
scores.append(mlp.score(X_test, y_test))

c:\Users\Paris\anaconda3\envs\desafio\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [20]:
from sklearn.ensemble import VotingClassifier
estimators = [('mlp', mlp), ('lgb', lgb), ('xgb', xgb)]
voting = VotingClassifier(estimators=estimators, voting='soft')

In [21]:
voting.fit(X_train,y_train)
y_pred = voting.predict(X_test)
accuracy_score(y_pred, y_test)

c:\Users\Paris\anaconda3\envs\desafio\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paris\anaconda3\envs\desafio\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.98095

In [22]:
scores.append(accuracy_score(y_pred, y_test))

In [23]:
names = ['Logistic', 'Decision Tree', 'Random Forest', 'AdaBoost', 'Gradient Boosting', 'CatBoost', 'Lightgbm', 'XGBoost', 'MLP', 'Voting']
pd.DataFrame(scores, index=names, columns=['Accuracy']).sort_values(by='Accuracy', ascending=False)

,Accuracy
CatBoost,0.98140
Lightgbm,0.98100
Voting,0.98095
Gradient Boosting,0.98085
XGBoost,0.98075
Logistic,0.98020
MLP,0.98010
AdaBoost,0.97680
Random Forest,0.87070
Decision Tree,0.52565


In [24]:
df_feature = pd.DataFrame(cat.get_feature_importance(), index=X.columns, columns=['feature_importance']).sort_values(by='feature_importance', ascending=False)
df_feature

,feature_importance
diff_alta,24.454314
age_18-35,21.182803
type_turistica,10.948242
type_historica,10.583021
comp_solo,10.049031
type_patrimonio,9.400540
diff_baja,5.735134
type_literaria,2.391557
time,1.913506
trans_a pie,1.606346
